<a href="https://colab.research.google.com/github/dudaholandah/PIBIC/blob/main/corgis_visualization_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Visualization of the Corgis DataSet

Instalação do plotly e do renderer Orca, para que o Github apresente as imagens geradas pelo Plotly

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp "/content/drive/My Drive/food/CORGIS_dataset.csv" "corgis.csv"
!cp "/content/drive/My Drive/food/australian_AUSNUT_2011_Food_Nutrient_Database.xls" "australian.xls"

In [ ]:
!pip install plotly==4.14.3
!pip install orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
!pip install umap-learn
!pip install trimap

     |████████████████████████████████| 646 kB 6.5 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391671 sha256=89feb0cf8bb16048c42f92b1735ee8f3f1d7ae567d2b78829e7b8fe1ac4dc791
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


Bibliotecas importantes

In [ ]:
import pandas as pd
from sklearn import preprocessing
#from ipywidgets import interactive, HBox, VBox
import plotly.express as px
#from plotly.offline import init_notebook_mode, iplot
##import plotly.offline as py
import plotly.graph_objects as go
import numpy as np
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.manifold import TSNE
from sklearn.manifold import Isomap
from sklearn.manifold import trustworthiness
from umap import UMAP
import trimap

Abre o arquivo .data utilizando o Pandas. O conjunto auto-mpg pode ser baixado aqui:


https://archive.ics.uci.edu/ml/datasets/Auto+MPG

In [ ]:
raw_data = pd.read_csv("corgis.csv")

In [ ]:
raw_data.head()

,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,Data.Fiber,...,Data.Major Minerals.Phosphorus,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
0,Milk,"Milk, human",11000000,0,7,0,6.89,14,16.0,0.0,...,14,51,17,0.17,61,0.05,0.011,5.0,0.08,0.3
1,Milk,"Milk, NFS",11100000,0,4,0,4.87,8,17.9,0.0,...,103,157,39,0.42,59,0.56,0.060,0.1,0.03,0.2
2,Milk,"Milk, whole",11111000,0,7,0,4.67,12,17.8,0.0,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3
3,Milk,"Milk, low sodium, whole",11111100,0,7,0,4.46,14,16.0,0.0,...,86,253,3,0.38,29,0.36,0.034,0.9,0.08,0.3
4,Milk,"Milk, calcium fortified, whole",11111150,0,7,0,4.67,12,17.8,0.0,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3


In [ ]:
print(f'Number of Data instances: {raw_data.shape[0]}')
print(f'Number of Data Attributes: {raw_data.shape[1]}')

Number of Data instances: 7083
Number of Data Attributes: 38


Labels information

In [ ]:
print(raw_data['Category'].value_counts())

Infant formula                          173
Rice                                    143
Bread                                   119
Potato                                  105
Cookie                                  100
                                       ... 
Layer dip                                 1
Bean dip                                  1
Refried beans with meat                   1
Black bean salad                          1
Industrial oil as ingredient in food      1
Name: Category, Length: 2429, dtype: int64


Remoção de instâncias com valores ausentes:

In [ ]:
raw_data = raw_data.replace('?',np.NaN)
raw_data = raw_data.replace('NA',np.NaN)
raw_data = raw_data.dropna(axis=0)

Remoção do atributo irrelevante para descoberta de conhecimento "Nutrient Data Bank Number" e remoção do atributo "Description" que é uma lista de ingredientes (strings) -- análise considera apenas os nutrientes

In [ ]:
raw_data = raw_data.drop(['Nutrient Data Bank Number'],axis=1)
raw_data = raw_data.drop(['Description'],axis=1)

In [ ]:
raw_data.head()

,Category,Data.Alpha Carotene,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,Data.Fiber,Data.Lutein and Zeaxanthin,Data.Lycopene,...,Data.Major Minerals.Phosphorus,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
0,Milk,0,7,0,6.89,14,16.0,0.0,0,0,...,14,51,17,0.17,61,0.05,0.011,5.0,0.08,0.3
1,Milk,0,4,0,4.87,8,17.9,0.0,0,0,...,103,157,39,0.42,59,0.56,0.060,0.1,0.03,0.2
2,Milk,0,7,0,4.67,12,17.8,0.0,0,0,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3
3,Milk,0,7,0,4.46,14,16.0,0.0,0,0,...,86,253,3,0.38,29,0.36,0.034,0.9,0.08,0.3
4,Milk,0,7,0,4.67,12,17.8,0.0,0,0,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3


Filtragem

Vamos analisa somente os produtos cujas categorias indiquem a presença de leite



In [ ]:
flag_milk = []
for label in raw_data['Category']:
    if "milk" in label or "Milk" in label:
        flag_milk.append(True)
    else:
        flag_milk.append(False)

In [ ]:
df_milk = raw_data[flag_milk]

In [ ]:
print(f'Number of Data instances: {df_milk.shape[0]}')
print(f'Number of Data Attributes: {df_milk.shape[1]}')

Number of Data instances: 129
Number of Data Attributes: 36


Pré-processamento


In [ ]:
labels = df_milk['Category'].values
df_milk = df_milk.drop(['Category'],axis=1)
X = df_milk.values

Normalização dos atributos para o intervalo [0,1]

In [ ]:
normalize = preprocessing.MinMaxScaler()
xscaled = normalize.fit_transform(X)

In [ ]:
xscaled

array([[0.        , 0.12727273, 0.        , ..., 0.09174312, 0.00651997,
        0.05263158],
       [0.        , 0.07272727, 0.        , ..., 0.00183486, 0.00244499,
        0.03508772],
       [0.        , 0.12727273, 0.        , ..., 0.        , 0.00407498,
        0.05263158],
       ...,
       [0.        , 0.05454545, 0.        , ..., 0.        , 0.00244499,
        0.03508772],
       [0.        , 0.05454545, 0.        , ..., 0.        , 0.00325998,
        0.03508772],
       [0.        , 0.        , 0.        , ..., 0.55045872, 0.57620212,
        0.0877193 ]])

## Visualização baseada no posicionamento de pontos utilizando Análise de Componentes Principais

Reduz a dimensionalidade dos dados

In [ ]:
sklearn_pca = sklearnPCA(n_components=2)
X_pca = sklearn_pca.fit_transform(xscaled)

#pca = PCA(n_components=2)
#components = pca.fit_transform(X)

fig = px.scatter(X_pca, x=0, y=1, color=labels.astype(object))
fig.show()

## UMAP

In [ ]:
umap_2d = UMAP(n_components=2, init='random', random_state=0)
X_umap = umap_2d.fit_transform(xscaled)

fig = px.scatter(X_umap, x=0, y=1, color=labels.astype(object))
fig.show()

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning:

The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.



## t-Stochastic Distributed Neighbor Embedding Projection (t-SNE)


Vamos gerar um gráfico de linhas para o conjunto de dados Auto-MPG Cars. Considere que cada carro é enumerado por um número inteiro de 1 a N, em que N é a quantidade de instâncias. Agora vamos criar uma nova coluna no DataFrame para possibilitar seu uso como eixo X no gráfico de linhas (Line Chart) do Plotly:

In [ ]:
tsne = TSNE(n_components=2,perplexity=5,learning_rate=100.0,metric='euclidean', init='random')
X_tsne = tsne.fit_transform(xscaled)

fig = px.scatter(X_tsne, x=0, y=1, color=labels.astype(object))
fig.show()


## TriMap

Triplets Mapping (TriMap) is a dimensionality reduction method that uses triplet constraints to form a low-dimensional embedding of a set of points. The triplet constraints are of the form “point i is closer to point j than point k”. The triplets are sampled from the high-dimensional representation of the points and a weighting scheme is used to reflect the importance of each triplet.

https://github.com/eamid/trimap


In [ ]:
X_trimap = trimap.TRIMAP().fit_transform(xscaled)

fig = px.scatter(X_trimap, x=0, y=1, color=labels.astype(object))
fig.show()

## Avaliação da Qualidade de Projeções

### Neighborhood Preservation

In [ ]:
kneigh = []
y_tsne = []
y_pca = []
y_umap = []

for k in range(1,51):
    precision = trustworthiness(xscaled, X_tsne, n_neighbors=k, metric='euclidean' )
    kneigh.append(k)
    y_tsne.append(precision)
    precision = trustworthiness(xscaled, X_pca, n_neighbors=k, metric='euclidean')
    y_pca.append(precision)
    precision = trustworthiness(xscaled, X_umap, n_neighbors=k, metric='euclidean')
    y_umap.append(precision)


Em seguida, podemos criar um gráfico de linhas:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=kneigh, y=y_tsne,
                    mode='lines+markers',
                    name='t-SNE'))
fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                    mode='lines+markers',
                    name='PCA'))
fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                    mode='lines+markers',
                    name='UMAP'))

fig.show()

### Coeficiente de Silhueta

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans

clusterer = KMeans(n_clusters=10, random_state=10)
cluster_labels_high = clusterer.fit_predict(xscaled)

cluster_labels_low = clusterer.fit_predict(X_tsne)

silhouette_avg_high = silhouette_score(X, cluster_labels_high)

silhouette_avg_low = silhouette_score(X_tsne, cluster_labels_low)

print(silhouette_avg_high)
print(silhouette_avg_low)

0.11493075122390936
0.5937349


In [ ]:
clusterer = KMeans(n_clusters=10, random_state=10)
cluster_labels_high = clusterer.fit_predict(xscaled)

cluster_labels_low = clusterer.fit_predict(X_umap)

silhouette_avg_high = silhouette_score(X, cluster_labels_high)

silhouette_avg_low = silhouette_score(X_umap, cluster_labels_low)

print(silhouette_avg_high)
print(silhouette_avg_low)

0.11493075122390936
0.5837925


In [ ]:
clusterer = KMeans(n_clusters=10, random_state=10)
cluster_labels_high = clusterer.fit_predict(xscaled)

cluster_labels_low = clusterer.fit_predict(X_pca)

silhouette_avg_high = silhouette_score(X, cluster_labels_high)

silhouette_avg_low = silhouette_score(X_pca,cluster_labels_low)

print(silhouette_avg_high)
print(silhouette_avg_low)

0.11493075122390936
0.375094816195165


## Global Score

https://github.com/eamid/trimap

In [ ]:
gs = trimap.TRIMAP(verbose=False).global_score(xscaled, X_trimap)
print(f"TriMap global score {gs}")

TriMap global score 0.2455117491079787


In [ ]:
gs = trimap.TRIMAP(verbose=False).global_score(xscaled, X_tsne)
print(f"tSNE global score {gs}")

tSNE global score 0.23552576869253314


In [ ]:
gs = trimap.TRIMAP(verbose=False).global_score(xscaled, X_umap)
print(f"UMAP global score {gs}")

UMAP global score 0.25658366986251674


In [ ]:
gs = trimap.TRIMAP(verbose=False).global_score(xscaled, X_pca)
print(f"PCA global score {gs}")

PCA global score 1.0
